# Fine-tuning `BertForQuestionAnswering` on SQuAD 2.0
Pavlos Spanoudakis (sdi1800184)

Colab does not have `transformers` pre-installed

In [ ]:
!pip install transformers

### Setting up hyperparameters

In [ ]:
LEARNING_RATE=5e-5
EPOCHS=3
BATCH_SIZE=16

MAX_INPUT_LENGTH=400
MAX_CONTEXT_LENGTH=350

### Setting up Dataset paths

In [ ]:
TRAIN_SET_PATH = '../input/squad-20/train-v2.0.json'
DEV_SET_PATH = '../input/squad-20/dev-v2.0.json'

### Necessary Imports

In [ ]:
import json
from typing import List, Dict, Tuple, Iterable
from numbers import Number

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Optimizer, Adam

from transformers import BertTokenizer, BertForQuestionAnswering
from transformers.tokenization_utils_base import PaddingStrategy, TruncationStrategy

### Enabling GPU acceleration

In [ ]:
CPU_DEVICE = 'cpu'
CUDA_DEVICE = 'cuda'
DEVICE = CUDA_DEVICE if torch.cuda.is_available() else CPU_DEVICE

### Storing context & questions information

In [ ]:
NO_ANSWER = (0, 0)

class QuestionContext:
    """ Represents a SQuAD Context, along with any useful information. """
    def __init__(self, text: str, tokenizer: BertTokenizer) -> None:
        self.text = text
        self.tokenIds = tokenizer(text, truncation=True, max_length=MAX_CONTEXT_LENGTH)['input_ids']
        self.tokens = tokenizer.convert_ids_to_tokens(self.tokenIds)
        whitespaces = []
        for i, c in enumerate(text):
            if c == ' ':
                whitespaces.append(i)
        
        self.whitespaces = tuple(whitespaces)

    def getAnswerTokenIndexes(self, startCharIndex: int, endCharIndex: int) -> Tuple[int, int]:
        """ Maps the given character indexes to context token indexes. """
        answerStart = -1
        answerEnd = -1
        currChar = 0
        for index, token in enumerate(self.tokens):
            if (index != 0) and (index != len(self.tokens) - 1):
                cleanToken = token.replace('##', '')
                for c in cleanToken:
                    if currChar == startCharIndex:
                        answerStart = index
                    if currChar == endCharIndex:
                        answerEnd = index
                        return (answerStart, answerEnd)
                    currChar += 1
        return NO_ANSWER

class Question:
    """ Represents a SQuAD Question, along with any useful information. """
    def __init__(self, text: str, answer: Dict[str, int], context: QuestionContext, isImpossible = False) -> None:
        # Original question text
        self.text = text
        # Reference to the QuestionContext object
        self.context = context
        
        # Storing the answer start and end token indexes
        if isImpossible:
            self.answer = NO_ANSWER
        else:
            endCharIndex = answer['answer_start'] + len(answer['text']) - 1
            whitespacesBeforeAnswer = 0
            whitespacesInAnswer = 0
            for i in context.whitespaces:
                if i >= answer['answer_start']:
                    if i < endCharIndex:
                        whitespacesInAnswer += 1
                    else:
                        break
                else:
                    whitespacesBeforeAnswer += 1
            noWhitespaceStart = answer['answer_start'] - whitespacesBeforeAnswer
            noWhitespaceEnd = noWhitespaceStart + len(answer['text']) - 1 - whitespacesInAnswer
            self.answer = context.getAnswerTokenIndexes(noWhitespaceStart, noWhitespaceEnd)

    def __repr__(self) -> str:
        if self.answer == NO_ANSWER:            
            answer = ' '.join(self.context.tokens[self.answer[0]:self.answer[1]+1])
        else:
            answer = ''
        return str({
            "text": self.text,
            "answer_start": self.answer[0],
            "answer_end": self.answer[1],
            "answer": answer
        })

### Reading and storing each Dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def readDataset(path: str, limit: int=None) -> List[Question]:
    # Question objects will be stored here
    questions = []
    with open(path) as samplesFile:
        samplesRaw = json.load(samplesFile)['data']
        for group in samplesRaw:
            for paragraph in group['paragraphs']:
                context = QuestionContext(paragraph['context'], tokenizer)
                for qa in paragraph['qas']:
                    answer = qa['answers'][0] if not qa['is_impossible'] else None
                    questions.append(Question(qa['question'], answer, context, qa['is_impossible']))
                    if limit and len(questions) >= limit:
                        return questions

    return questions

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

### `Dataset` class to provide to a `DataLoader`

In [ ]:
class QuestionsDataset(torch.utils.data.Dataset):
    def __init__(self, questions: List[Question]) -> None:
        super().__init__()
        self.questions = [q.text for q in questions]
        self.contexts = [q.context.text for q in questions]
        self.answers = [torch.tensor(q.answer) for q in questions]

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, index: int):
        return self.questions[index], self.contexts[index], self.answers[index]

### Routines for metrics calculation during training

In [ ]:
def predictionsF1Score(modelAnswers: Iterable[Tuple[int]], trueAnswers: Iterable[Tuple[int]]) -> Number:

    def findAnswerF1(modelAnswer: Tuple[int], trueAnswer: Tuple[int]) -> Number:
        modelSequence = range(modelAnswer[0], modelAnswer[1] + 1 )
        trueSequence = range(trueAnswer[0], trueAnswer[1] + 1 )
        numCommon = len(set(trueSequence).intersection(modelSequence))

        if numCommon == 0:
            return 0
        
        precision = 1.0 * numCommon / len(trueSequence)
        recall = 1.0 * numCommon / len(modelSequence)
        f1 = (2 * precision * recall) / (precision + recall)
        return f1

    totalF1 = 0
    for model, gold in zip(modelAnswers, trueAnswers):
        totalF1 += findAnswerF1(model, gold)

    return totalF1/len(trueAnswers)

def predictionsExactScore(modelAnswers: Iterable[Tuple[int]], trueAnswers: Iterable[Tuple[int]]) -> Number:
    correct = 0
    for model, true in zip(modelAnswers, trueAnswers):
        correct += int( (model[0] == true[0]) and (model[1] == true[1]) )
    
    return correct/len(trueAnswers)

def getPredictedAnswers(startLogits: torch.Tensor, endLogits: torch.Tensor) -> Tuple[np.ndarray, np.ndarray]:
    softmaxStart = torch.log_softmax(startLogits, dim = 1)
    _, start = torch.max(softmaxStart, dim = 1)

    softmaxEnd = torch.log_softmax(endLogits, dim = 1)
    _, end = torch.max(softmaxEnd, dim = 1)
    return (start.cpu().detach().numpy(), end.cpu().detach().numpy())

### Training the model & displaying metrics after each epoch

In [ ]:
def trainModel(model: BertForQuestionAnswering, optimizer: Optimizer, tokenizer: BertTokenizer,
               trainSetLoader: DataLoader, devSetLoader: DataLoader) -> None:
    for epoch in range(EPOCHS):
        trainExactScores = []
        trainLosses = []
        trainF1 = []
        devExactScores = []
        devLosses = []
        devF1 = []
        
        model.train()        
        for batchQuestions, batchContexts, batchAnswers in trainSetLoader:
            qaPairs = [[question, answer] for question, answer in zip(batchQuestions, batchContexts)]
            tok = tokenizer._batch_encode_plus( qaPairs,
                                                truncation_strategy=TruncationStrategy.ONLY_SECOND,
                                                max_length=MAX_INPUT_LENGTH,
                                                padding_strategy=PaddingStrategy.MAX_LENGTH,
                                                return_tensors="pt")
            inputIds = tok['input_ids'].to(DEVICE)
            segmentIds = tok['token_type_ids'].to(DEVICE)
            attentionMask = tok['attention_mask'].to(DEVICE)
            startPositions = batchAnswers[:, 0].to(DEVICE)
            endPositions = batchAnswers[:, 1].to(DEVICE)

            outputs = model(input_ids=inputIds, token_type_ids=segmentIds, attention_mask=attentionMask, start_positions=startPositions, end_positions=endPositions)    

            optimizer.zero_grad()

            outputs.loss.backward()

            optimizer.step()

            startPredictions, endPredictions = getPredictedAnswers(outputs.start_logits, outputs.end_logits)
            modelAnswers = np.vstack((startPredictions, endPredictions)).T

            trainExactScores.append(predictionsExactScore(modelAnswers, batchAnswers))
            trainLosses.append(outputs.loss.item())
            trainF1.append(predictionsF1Score(modelAnswers, batchAnswers))
            
        model.eval()
        with torch.no_grad():
            for batchQuestions, batchContexts, batchAnswers in devSetLoader:
                qaPairs = [[question, answer] for question, answer in zip(batchQuestions, batchContexts)]
                tok = tokenizer._batch_encode_plus( qaPairs,
                                                    truncation_strategy=TruncationStrategy.ONLY_SECOND,
                                                    max_length=MAX_INPUT_LENGTH,
                                                    padding_strategy=PaddingStrategy.MAX_LENGTH,
                                                    return_tensors="pt")                
                inputIds = tok['input_ids'].to(DEVICE)
                segmentIds = tok['token_type_ids'].to(DEVICE)
                attentionMask = tok['attention_mask'].to(DEVICE)
                startPositions = batchAnswers[:, 0].to(DEVICE)
                endPositions = batchAnswers[:, 1].to(DEVICE)
                
                outputs = model(input_ids=inputIds, token_type_ids=segmentIds,
                                attention_mask=attentionMask,
                                start_positions=startPositions, end_positions=endPositions)
                
                startPredictions, endPredictions = getPredictedAnswers(outputs.start_logits, outputs.end_logits)
                modelAnswers = np.vstack((startPredictions, endPredictions)).T

                devExactScores.append(predictionsExactScore(modelAnswers, batchAnswers))
                devLosses.append(outputs.loss.item())
                devF1.append(predictionsF1Score(modelAnswers, batchAnswers))

        print(f"################ EPOCH {epoch} ################")
        print("--------------- Train Set ---------------")
        print(f"Exact: {sum(trainExactScores)/len(trainExactScores):.5f}", end=' ')
        print(f"F1: {sum(trainF1)/len(trainF1):.5f}", end=' ')
        print(f"Loss: {sum(trainLosses)/len(trainLosses):.5f}")
        print("------------- Validation Set ------------")
        print(f"Exact: {sum(devExactScores)/len(devExactScores):.5f}", end=' ')
        print(f"F1: {sum(devF1)/len(devF1):.5f}", end=' ')
        print(f"Loss: {sum(devLosses)/len(devLosses):.5f}")

## Execution Flow

In [ ]:
trainQuestions = readDataset(TRAIN_SET_PATH)
devQuestions = readDataset(DEV_SET_PATH)

trainSetLoader = DataLoader(QuestionsDataset(trainQuestions), batch_size=BATCH_SIZE, shuffle=False)
devSetLoader = DataLoader(QuestionsDataset(devQuestions), batch_size=BATCH_SIZE, shuffle=False)

model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
trainModel(model, optimizer, tokenizer, trainSetLoader, devSetLoader)

################ EPOCH 0 ################
--------------- Train Set ---------------
Exact: 0.30985 F1: 0.39476 Loss: 2.59632
------------- Validation Set ------------
Exact: 0.50463 F1: 0.50680 Loss: 3.06969
################ EPOCH 1 ################
--------------- Train Set ---------------
Exact: 0.37853 F1: 0.55488 Loss: 1.72829
------------- Validation Set ------------
Exact: 0.50698 F1: 0.54050 Loss: 2.38843
################ EPOCH 2 ################
--------------- Train Set ---------------
Exact: 0.47340 F1: 0.66173 Loss: 1.31572
------------- Validation Set ------------
Exact: 0.52195 F1: 0.57065 Loss: 2.24974
